# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
#To show multiple outputs and misc. changes
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # Show all results without print (last_expr)

In [2]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [5]:
# filepath = "data/song_data"
# len(get_files(filepath))

def process_data(filepath):
    # get all files matching extension from directory
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
        print(root)

    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))
    print(filepath)
process_data("data/log_data")

data/log_data
data/log_data\2018
data/log_data\2018\11
30 files found in data/log_data
data/log_data


# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
# Get file paths
filepath = "data/song_data"
song_files = get_files(filepath)

# Load single file to dataframe
df = pd.read_json(song_files[0], typ="series")
df = pd.DataFrame([df])
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,None,None,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
# Load data
df_songs_data = pd.DataFrame(columns=["num_songs", "artist_id", "artist_latitude", "artist_longitude", "artist_location", "artist_name", "song_id", "title", "duration", "year"])

# Load data from files to dataframe
for i in range(len(song_files)):
    df = pd.read_json(song_files[i], typ="series")
    df = pd.DataFrame([df])
    df_songs_data = df_songs_data.append(df)
df_songs_data = df_songs_data.sort_values(by="song_id").reset_index(drop=True)

# Fix columns
df_songs_data["num_songs"] = df_songs_data["num_songs"].astype(int)
df_songs_data["artist_id"] = df_songs_data["artist_id"].astype(str)
df_songs_data["artist_latitude"] = df_songs_data["artist_latitude"].astype(float)
df_songs_data["artist_longitude"] = df_songs_data["artist_longitude"].astype(float)
df_songs_data["artist_location"] = df_songs_data["artist_location"].replace(r'', np.NaN)
df_songs_data["artist_location"] = df_songs_data["artist_location"].astype(str)
df_songs_data["artist_name"] = df_songs_data["artist_name"].astype(str)
df_songs_data["song_id"] = df_songs_data["song_id"].astype(str)
df_songs_data["title"] = df_songs_data["title"].astype(str)
df_songs_data["duration"] = df_songs_data["duration"].astype(str)
# df_songs_data["year"] = df_songs_data["year"].replace(0, np.NaN)
df_songs_data["year"] = df_songs_data["year"].astype(int)

df_songs_data.head(n=3)
df_songs_data.info()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARPBNLO1187FB3D52F,40.71455,-74.00712,"New York, NY",Tiny Tim,SOAOIBZ12AB01815BE,I Hold Your Hand In Mine [Live At Royal Albert...,43.36281,2000
1,1,ARDR4AC1187FB371A1,NaN,NaN,nan,Montserrat Caballé;Placido Domingo;Vicente Sar...,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,511.16363,0
2,1,ARMAC4T1187FB3FA4C,40.82624,-74.47995,"Morris Plains, NJ",The Dillinger Escape Plan,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,207.77751,2004


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   num_songs         71 non-null     int32  
 1   artist_id         71 non-null     object 
 2   artist_latitude   31 non-null     float64
 3   artist_longitude  31 non-null     float64
 4   artist_location   71 non-null     object 
 5   artist_name       71 non-null     object 
 6   song_id           71 non-null     object 
 7   title             71 non-null     object 
 8   duration          71 non-null     object 
 9   year              71 non-null     int32  
dtypes: float64(2), int32(2), object(6)
memory usage: 5.1+ KB


In [8]:
song_data = list(df_songs_data[["song_id", "title", "artist_id", "year", "duration"]].values[0])
song_data

['SOAOIBZ12AB01815BE',
 'I Hold Your Hand In Mine [Live At Royal Albert Hall]',
 'ARPBNLO1187FB3D52F',
 2000,
 '43.36281']

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
for i, row in df_songs_data[["song_id", "title", "artist_id", "year", "duration"]].iterrows():
    cur.execute(song_table_insert, list(row))
    conn.commit()
    
# cur.execute(song_table_insert, song_data)
# conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
artist_data = list(df_songs_data[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values[0])
artist_data

['ARPBNLO1187FB3D52F', 'Tiny Tim', 'New York, NY', 40.71455, -74.00712]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
for i, row in df_songs_data[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].iterrows():
    cur.execute(artist_table_insert, list(row))
    conn.commit()

# cur.execute(artist_table_insert, artist_data)
# conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
# Get file paths
filepath = "data/log_data"
log_files = get_files(filepath)

# Load single file to dataframe
df = pd.read_json(log_files[0], lines=True)
df.head(n=3)
df.columns

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [13]:
# Load data
df_log_data = pd.DataFrame(columns=['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'method', 'page','registration', 'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'])

for i in range(len(log_files)):
    df = pd.read_json(log_files[i], lines=True)
    df_log_data = df_log_data.append(df)
df_log_data = df_log_data.sort_values(by="ts").reset_index(drop=True)

# Fix columns
df_log_data["length"] = df_log_data["length"].astype(str)
df_log_data["registration"] = df_log_data["registration"].astype(float)
df_log_data["sessionId"] = df_log_data["sessionId"].astype(int)
df_log_data["status"] = df_log_data["status"].astype(int)
# df_log_data["ts"] = df_log_data["ts"].astype(int)
df_log_data = df_log_data.sort_values(by="ts").reset_index(drop=True)
df_log_data["userId"] = df_log_data["userId"].replace(r"", 0)
df_log_data["userId"] = df_log_data["userId"].astype(int)
df_log_data["ts"] = pd.to_datetime(df_log_data['ts'], unit='ms') # .dt.tz_localize('UTC').dt.tz_convert('America/New_York')
df_log_data = df_log_data[df_log_data["page"]=="NextSong"]
df_log_data.head(n=3)
df_log_data.info()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,2018-11-01 21:01:46.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,2018-11-01 21:05:52.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Quem Quiser Encontrar O Amor,200,2018-11-01 21:08:16.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6820 entries, 1 to 8055
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   artist         6820 non-null   object        
 1   auth           6820 non-null   object        
 2   firstName      6820 non-null   object        
 3   gender         6820 non-null   object        
 4   itemInSession  6820 non-null   object        
 5   lastName       6820 non-null   object        
 6   length         6820 non-null   object        
 7   level          6820 non-null   object        
 8   location       6820 non-null   object        
 9   method         6820 non-null   object        
 10  page           6820 non-null   object        
 11  registration   6820 non-null   float64       
 12  sessionId      6820 non-null   int32         
 13  song           6820 non-null   object        
 14  status         6820 non-null   int32         
 15  ts             6820 n

In [14]:
time_df = df_log_data[["ts"]].copy()
time_df.columns = ["start_time"]
time_df["hour"] = time_df["start_time"].dt.hour
time_df["day"] = time_df["start_time"].dt.day
time_df["week"] = time_df["start_time"].dt.isocalendar().week
time_df["month"] = time_df["start_time"].dt.month
time_df["year"] = time_df["start_time"].dt.year
time_df["weekday"] = time_df["start_time"].dt.weekday
#t["day_name"] = t["start_time"].dt.day_name()
time_df = time_df.sort_values(by="start_time").reset_index(drop=True)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-01 21:01:46.796,21,1,44,11,2018,3
1,2018-11-01 21:05:52.796,21,1,44,11,2018,3
2,2018-11-01 21:08:16.796,21,1,44,11,2018,3
3,2018-11-01 21:11:13.796,21,1,44,11,2018,3
4,2018-11-01 21:17:33.796,21,1,44,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [15]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [16]:
user_df = df_log_data[["userId", "firstName", "lastName", "gender", "level"]].copy()
user_df.columns = ["user_id", "first_name", "last_name", "gender", "level"]
# user_df = user_df.drop_duplicates()
user_df.head(n=3)
user_df.info()

,user_id,first_name,last_name,gender,level
1,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free
5,8,Kaylee,Summers,F,free


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6820 entries, 1 to 8055
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     6820 non-null   int32 
 1   first_name  6820 non-null   object
 2   last_name   6820 non-null   object
 3   gender      6820 non-null   object
 4   level       6820 non-null   object
dtypes: int32(1), object(4)
memory usage: 293.0+ KB


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [17]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [18]:
# df_songs_data.head()
# df_log_data.head()
# df_songs_data.merge(df_log_data, how="inner", left_on=["artist_name", "title", "duration"], right_on=["artist", "song", "length"])

In [19]:
for index, row in df_log_data.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = [row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent]
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [20]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.